In [ ]:
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install matplotlib

In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import brier_score_loss, roc_auc_score

import matplotlib.pyplot as plt

# For nicer printing
pd.set_option("display.max_columns", 50)

In [ ]:
# Load CIMIS data 

DATA_DIR = "./cimis-hourly-data-multiple-stations/"

def file_to_df(file_name: str):
    # Load CSV file into a pandas DataFrame
    return pd.read_csv(file_name)

def get_datasets():
    # Uses shell commands, works in Jupyter
    !touch buffer.tmp
    !(ls $DATA_DIR | grep "csv") > buffer.tmp
    name_to_data = dict()  # Dictionary for Name to Data
    # Read the buffer.tmp and create Name to Data pairs
    with open("buffer.tmp", "r") as file:
        contents = [f for f in file.read().split("\n") if f]
        for content in contents:
            file_name = content.split(".")[0]
            if "all" not in file_name:
                name_to_data[file_name] = file_to_df(DATA_DIR + content)
    !rm -f buffer.tmp
    return name_to_data

datasets = get_datasets()

# Individual dataset names (one per station, e.g. '80-fresnostate')
dataset_names = [d for d in datasets.keys() if "all" not in d]

# Remove columns that contain "qc" using one sample dataset
sample_name = dataset_names[0]
columns_to_remove = [col for col in datasets[sample_name].columns.tolist() if "qc" in col]

for name in datasets:
    datasets[name].drop(columns=columns_to_remove, inplace=True)

# Combine into a single DataFrame, ensuring 'Stn Id' exists
dfs = []
for name, d in datasets.items():
    df_station = d.copy()
    if "Stn Id" not in df_station.columns:
        # If the CSV does not include the station ID column, create one from the key
        df_station["Stn Id"] = name
    dfs.append(df_station)

df = pd.concat(dfs, ignore_index=True)

print("Combined df shape:", df.shape)
print(df.columns)
df.head()

In [5]:
# Build Datetime and frost_now with handling for 24:00 ---

TEMP_COL = "Air Temp (C)"   # adjust if your column is named differently

# Extract hour from integers like 100, 1300, 2400
raw_hours = df["Hour (PST)"].astype(int)

# hour_int = 0–23 normally, but 24 → 0 of next day
df["Hour_int"] = (raw_hours // 100)

# Identify rows where the hour is 24
mask_24 = df["Hour_int"] == 24

# Set hour=0 for parsing
df.loc[mask_24, "Hour_int"] = 0

# Parse date with hour=0 (even for original 24:00 rows)
df["Datetime"] = pd.to_datetime(
    df["Date"] + " " + df["Hour_int"].astype(str).str.zfill(2) + ":00",
    format="%m/%d/%Y %H:%M"
)

# For rows that were originally 24:00 → add 1 day
df.loc[mask_24, "Datetime"] = df.loc[mask_24, "Datetime"] + pd.Timedelta(days=1)

# Final sorting
df = df.sort_values(["Stn Id", "Datetime"]).reset_index(drop=True)

# Create frost flag
df["frost_now"] = (df[TEMP_COL] < 0).astype(int)

df[["Stn Id", "Date", "Hour (PST)", "Datetime", TEMP_COL, "frost_now"]].head(10)

,Stn Id,Date,Hour (PST),Datetime,Air Temp (C),frost_now
0,2,9/28/2010,100,2010-09-28 01:00:00,20.4,0
1,2,9/28/2010,200,2010-09-28 02:00:00,20.0,0
2,2,9/28/2010,300,2010-09-28 03:00:00,18.5,0
3,2,9/28/2010,400,2010-09-28 04:00:00,18.0,0
4,2,9/28/2010,500,2010-09-28 05:00:00,17.4,0
5,2,9/28/2010,600,2010-09-28 06:00:00,16.9,0
6,2,9/28/2010,700,2010-09-28 07:00:00,17.3,0
7,2,9/28/2010,800,2010-09-28 08:00:00,22.1,0
8,2,9/28/2010,900,2010-09-28 09:00:00,26.0,0
9,2,9/28/2010,1000,2010-09-28 10:00:00,28.8,0


In [9]:
# Build frost labels for multiple horizons

HORIZONS = [3, 6, 12, 24]

# Make sure temperature is numeric
df[TEMP_COL] = pd.to_numeric(df[TEMP_COL], errors="coerce")

def add_frost_labels(df, horizons, temp_col="Air Temp (C)"):
    df = df.copy()
    df = df.sort_values(["Stn Id", "Datetime"]).reset_index(drop=True)
    
    for H in horizons:
        label_col = f"frost_next_{H}h"
        labels = []

        for stn_id, g in df.groupby("Stn Id", sort=False):
            temps = g[temp_col].to_numpy()
            n = len(temps)
            lab = np.zeros(n, dtype=int)

            for i in range(n):
                j_end = min(n, i + H + 1)   # exclusive
                window = temps[i+1:j_end]   # (t, t+H]

                # Keep only non-NaN values
                window_valid = window[~np.isnan(window)]

                if len(window_valid) > 0 and window_valid.min() < 0:
                    lab[i] = 1
                else:
                    lab[i] = 0

            labels.append(pd.Series(lab, index=g.index, name=label_col))

        labels = pd.concat(labels).sort_index()
        df[label_col] = labels

    return df

df = add_frost_labels(df, HORIZONS, temp_col=TEMP_COL)
df[["Stn Id", "Datetime", TEMP_COL] + [f"frost_next_{h}h" for h in HORIZONS]].head()

,Stn Id,Datetime,Air Temp (C),frost_next_3h,frost_next_6h,frost_next_12h,frost_next_24h
0,2,2010-09-28 01:00:00,20.4,0,0,0,0
1,2,2010-09-28 02:00:00,20.0,0,0,0,0
2,2,2010-09-28 03:00:00,18.5,0,0,0,0
3,2,2010-09-28 04:00:00,18.0,0,0,0,0
4,2,2010-09-28 05:00:00,17.4,0,0,0,0


## Baseline Probabilities ##
Compute climatology, dew point, and persistence baselines for comparison 

In [10]:
df["Datetime"].min(), df["Datetime"].max()
# (Timestamp('2010-09-28 01:00:00'), Timestamp('2025-09-29 00:00:00'))
# Total ~ 15 years
# 80% point ~2022 
# 70% ~2018-19
# we want test to be atleast 3-5 frost seasons

(Timestamp('2010-09-28 01:00:00'), Timestamp('2025-09-29 00:00:00'))

In [11]:
# Time-based train / test split

CUTOFF_DATE = "2019-01-01"  # adjust as needed
# choose a cutoff about 70-80% through a dataset timeline
# e.g. for 2010-2025, 80% point ~= 2017
# you want a training cutoff early BEFORE the data you are testing on

train = df[df["Datetime"] < CUTOFF_DATE].copy()
test  = df[df["Datetime"] >= CUTOFF_DATE].copy()

len(train), len(test)

(1303326, 1064034)

In [12]:
# Climatology baseline
# For each horizon H:
# Compute P(frost_next_Hh | month) on train
# Apply to test, fallback to global rate if needed

for split in [train, test]:
    split["month"] = split["Datetime"].dt.month

for H in HORIZONS:
    label_col = f"frost_next_{H}h"
    clim_col  = f"p_clim_{H}h"

    global_rate = train[label_col].mean()
    monthly_rate = train.groupby("month")[label_col].mean()

    test[clim_col] = test["month"].map(monthly_rate)
    test[clim_col] = test[clim_col].fillna(global_rate)

test[["Datetime", "month"] + [f"p_clim_{h}h" for h in HORIZONS]].head()

,Datetime,month,p_clim_3h,p_clim_6h,p_clim_12h,p_clim_24h
72407,2019-01-01 00:00:00,1,0.05619,0.077182,0.117944,0.173984
72408,2019-01-01 01:00:00,1,0.05619,0.077182,0.117944,0.173984
72409,2019-01-01 02:00:00,1,0.05619,0.077182,0.117944,0.173984
72410,2019-01-01 03:00:00,1,0.05619,0.077182,0.117944,0.173984
72411,2019-01-01 04:00:00,1,0.05619,0.077182,0.117944,0.173984


In [16]:
# Dew-point empirical rule baseline 
# For each horizon H: 
# Bin dew point on train
# Use empirical P(frost_next_Hh | Td bin) as baseline probability

DEW_COL = "Dew Point (C)"  # adjust if your column name is different

dew_bins = np.arange(-20, 21, 2)  # -20 to +20 by 2°C, adjust if needed

for H in HORIZONS:
    label_col = f"frost_next_{H}h"
    dew_col   = f"p_dew_{H}h"

    # Bin dew point in train and test
    train["Td_bin"] = pd.cut(train[DEW_COL], bins=dew_bins, include_lowest=True)
    test["Td_bin"]  = pd.cut(test[DEW_COL],  bins=dew_bins, include_lowest=True)

    # Empirical P(frost | Td_bin) on train
    dew_table = train.groupby("Td_bin", observed=False)[label_col].mean()

    # Global frost rate as fallback
    global_rate = train[label_col].mean()

    # Map Td_bin → probability (this is a Series indexed by Td_bin categories)
    prob = test["Td_bin"].map(dew_table)

    # Make absolutely sure it's a float Series, not categorical
    prob = prob.astype("float64")

    # Fill NaNs (bins not seen in train) with global rate, still float
    prob = prob.fillna(float(global_rate))

    # Finally assign plain float values into the test column
    test[dew_col] = prob.values

# Clean up temporary bin column
train.drop(columns=["Td_bin"], inplace=True, errors="ignore")
test.drop(columns=["Td_bin"], inplace=True, errors="ignore")

test[[DEW_COL] + [f"p_dew_{h}h" for h in HORIZONS]].head()


,Dew Point (C),p_dew_3h,p_dew_6h,p_dew_12h,p_dew_24h
72407,-2.5,0.235690,0.284197,0.357198,0.450512
72408,-3.3,0.235690,0.284197,0.357198,0.450512
72409,-4.1,0.251292,0.306622,0.388028,0.486446
72410,-3.6,0.235690,0.284197,0.357198,0.450512
72411,-3.9,0.235690,0.284197,0.357198,0.450512


In [17]:
# Persistence baseline
# Very simple rule: 
# If freezing now -> P(frost_next_Hh) = 1
# Else -> 0

for H in HORIZONS:
    persist_col = f"p_persist_{H}h"
    test[persist_col] = test["frost_now"].astype(float)

test[["frost_now"] + [f"p_persist_{h}h" for h in HORIZONS]].head()

,frost_now,p_persist_3h,p_persist_6h,p_persist_12h,p_persist_24h
72407,1,1.0,1.0,1.0,1.0
72408,1,1.0,1.0,1.0,1.0
72409,1,1.0,1.0,1.0,1.0
72410,1,1.0,1.0,1.0,1.0
72411,1,1.0,1.0,1.0,1.0


In [19]:
# Save baseline probabilities (standalone use)
# Export a CSV with: 
# Stn Id, Datetime
# Labels frost_next_Hh
# Baselines: climatology, dew, persistence

baseline_cols = []
for H in HORIZONS:
    baseline_cols += [
        f"frost_next_{H}h",
        f"p_clim_{H}h",
        f"p_dew_{H}h",
        f"p_persist_{H}h",
    ]

baseline_df = test[["Stn Id", "Datetime"] + baseline_cols].copy()

baseline_path = "frost_baselines_test.csv"
baseline_df.to_csv(baseline_path, index=False)

baseline_path

'frost_baselines_test.csv'

## Compare ML Model vs Baselines ##
Assuming a CSV with ML model's probabilities in format: 
- Stn ID, Datetime
- p_model_3h, p_model_6h, p_model_12h, p_model_24h
(can adjust names) 

In [24]:
# Load baseline file 
baseline_df = pd.read_csv("frost_baselines_test.csv", parse_dates=["Datetime"])
baseline_df.head()

,Stn Id,Datetime,frost_next_3h,p_clim_3h,p_dew_3h,p_persist_3h,frost_next_6h,p_clim_6h,p_dew_6h,p_persist_6h,frost_next_12h,p_clim_12h,p_dew_12h,p_persist_12h,frost_next_24h,p_clim_24h,p_dew_24h,p_persist_24h
0,2,2019-01-01 00:00:00,1,0.05619,0.235690,1.0,1,0.077182,0.284197,1.0,1,0.117944,0.357198,1.0,1,0.173984,0.450512,1.0
1,2,2019-01-01 01:00:00,1,0.05619,0.235690,1.0,1,0.077182,0.284197,1.0,1,0.117944,0.357198,1.0,1,0.173984,0.450512,1.0
2,2,2019-01-01 02:00:00,1,0.05619,0.251292,1.0,1,0.077182,0.306622,1.0,1,0.117944,0.388028,1.0,1,0.173984,0.486446,1.0
3,2,2019-01-01 03:00:00,1,0.05619,0.235690,1.0,1,0.077182,0.284197,1.0,1,0.117944,0.357198,1.0,1,0.173984,0.450512,1.0
4,2,2019-01-01 04:00:00,0,0.05619,0.235690,1.0,0,0.077182,0.284197,1.0,0,0.117944,0.357198,1.0,1,0.173984,0.450512,1.0


In [ ]:
# ******* TO DO **********
# Change file name and column names to match your setup

model_preds_path = "my_model_preds.csv"  # TODO: replace
model_df = pd.read_csv(model_preds_path, parse_dates=["Datetime"])

model_df.head()

In [ ]:
# Merge

merged = baseline_df.merge(
    model_df,
    on=["Stn Id", "Datetime"],
    how="inner"
)

print(len(baseline_df), len(model_df), len(merged))
merged.head()

In [ ]:
# Metric helpers

def evaluate_one_horizon(df, H):
    label_col = f"frost_next_{H}h"

    models = {
        "climatology": f"p_clim_{H}h",
        "dew_point":   f"p_dew_{H}h",
        "persistence": f"p_persist_{H}h",
        "logistic":    f"p_logit_{H}h",
        "ML_model":    f"p_model_{H}h",  # EDIT if needed
    }

    results = []
    needed_cols = [label_col] + list(models.values())
    df_h = df[needed_cols].dropna()
    y_true = df_h[label_col].values

    for name, col in models.items():
        if col not in df_h.columns:
            continue

        y_prob = df_h[col].values
        brier = brier_score_loss(y_true, y_prob)

        if len(np.unique(y_true)) == 2:
            auc = roc_auc_score(y_true, y_prob)
        else:
            auc = np.nan

        results.append({
            "horizon": H,
            "model": name,
            "brier": brier,
            "roc_auc": auc,
            "n_samples": len(df_h),
        })

    return pd.DataFrame(results)

all_results = pd.concat(
    [evaluate_one_horizon(merged, H) for H in HORIZONS],
    ignore_index=True
)
all_results

In [ ]:
#print 

for H in HORIZONS:
    print(f"=== Horizon {H}h ===")
    display(
        all_results[all_results["horizon"] == H]
        .sort_values("brier")
        .reset_index(drop=True)
    )

In [ ]:
# Bar plots

for H in HORIZONS:
    sub = all_results[all_results["horizon"] == H].copy()
    if sub.empty:
        continue
    plt.figure()
    plt.bar(sub["model"], sub["brier"])
    plt.title(f"Brier score by model – {H}h horizon")
    plt.ylabel("Brier score (lower=better)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()